# 0. Descriptors

- In Python, a descriptor is an object attribute with `binding behavior,` which means that it has methods defined for getting, setting, and deleting the attribute. Descriptors are used to customize attribute access and provide more control over how attributes are managed in classes. 

- The key methods associated with descriptors are:
    - 1. `__get__(self, instance, owner)`: Called when the descriptor's attribute is accessed. Tt should return the value of the attribute. `instance` is the instance of the object that the attribute is accessed on, and `owner` is the class of the instance. 
    - 2. `__set__(self, instance, value)`: Called when the descriptor's attribute is set. It allows you to define the behavior when a value is assigned to the attribute. 
    - 3. `__delete__(self, instance)`: Called when the descriptor's attribute is deleted. It allows you to define the behavior when the attribute is deleted. 

## Example 1: Positive Age

Imagine you want to create a class where the attribute value should always be a non-negative integer such as age assigned to an individual. You can use a descriptor to enforce this constraint. 

In [1]:
class NonNegativeInteger:
    def __init__(self, name) -> None:
        self._name = name

    def __get__(self, instance, owner):
        return instance.__dict__[self._name]

    def __set__(self, instance, value):
        if not isinstance(value, int) or value <0:
            raise ValueError(f"{self._name} must be a non-negative integer")
        instance.__dict__[self._name] = value

class User:
    age = NonNegativeInteger("age")

    def __init__(self, name, age) -> None:
        self.name = name
        self.age = age

In [2]:
user1 = User('Janet', 25)
user1.age

25

In this example, the `User` class has an attribute `age` which is an instance of `NonNegativeInteger`. The descriptor ensures that the age attribute for any User instance is always a non-negative integer. 

In [3]:
try:
    user1.age = -5
except ValueError as e:
    print(e)

age must be a non-negative integer


## Example 2: Contact Information 

Imagine you need to validate and store contact information for a group of users. The contact information includes a phone number, an email address, and a mailing address. The descriptor ensures that each piece of information meets certain criteria. 

In [4]:
class ContactInformation:
    def __init__(self, name) -> None:
        self._name = name
        self._phone_number = ""
        self._email_address = ""
        self._mailing_address = ""

    def __get__(self, instance, owner) -> dict:
        return {
            "phone_number": self._phone_number,
            "email_address": self._email_address,
            "mailing_address": self._mailing_address
        }

    def __set__(self, instance, contact_info) -> None:
        self._validate_and_set("phone_number", contact_info.get("phone_number", ""))
        self._validate_and_set("email_address", contact_info.get("email_address", ""))
        self._validate_and_set("mailing_address", contact_info.get("mailing_address", ""))

    def _validate_and_set(self, attribute, value) -> None:
        if attribute == "phone_number" and not self._is_valid_phone_number(value):
            raise ValueError(f"Invalid phone number: {value}")
        if attribute == "email_address" and not self._is_valid_email(value):
            raise ValueError(f"Invalid email address: {value}")

        setattr(self, "_" + attribute, value)

    def _is_valid_phone_number(self, phone_number) -> bool:
        # Example validation: a valid phone number should be numeric and have 10 digits
        return phone_number.isdigit() and len(phone_number) == 10

    def _is_valid_email(self, email_address) -> bool:
        # Example validation: a valid email address should contain '@'
        return "@" in email_address



class Person:
    contact_info = ContactInformation("contact_info")

    def __init__(self, name, contact_info) -> None:
        self.name = name
        self.contact_info = contact_info


In [5]:
person = Person("John Doe", {
    "phone_number": "5129889999",
    "email_address": "john.doe@gmail.com",
    "mailing_address": "123 Main St. Austin, TX, 78799"
})

# Accessing and updating the contact information using the descriptor
person.contact_info

{'phone_number': '5129889999',
 'email_address': 'john.doe@gmail.com',
 'mailing_address': '123 Main St. Austin, TX, 78799'}

In [6]:
# Attempting to set an invalid phone number
try:
    person.contact_info = {
         "phone_number": "+15129999098"
    }
except ValueError as e:
    print(e)

Invalid phone number: +15129999098


Here is a detailed explanation of the code:

**class ContactInformation:**

This class defines a data structure to store and manage personal contact information. It includes methods for initializing, accessing, and validating contact details.

**__init__(self, name):**

The constructor initializes the contact information for a specific person. It takes the person's name as an argument and sets the private attributes for phone number, email address, and mailing address to empty strings.

**__get__(self, instance, owner):**

This method acts as a descriptor for the `contact_info` attribute of the `Person` class. When the `contact_info` attribute is accessed, it returns a dictionary containing the current values of the phone number, email address, and mailing address.

**__set__(self, instance, contact_info):**

This method is also a descriptor and is called when the `contact_info` attribute is assigned a new value. It takes the updated contact information as a dictionary and validates each field using the `_validate_and_set` method before updating the corresponding private attributes.

**_validate_and_set(self, attribute, value):**

This private method validates the value of a specific contact information attribute (phone number, email address, or mailing address) before updating it. It performs basic checks for the format and content of the value and raises a `ValueError` if it's invalid.

**_is_valid_phone_number(self, phone_number):**

This private method checks if the provided phone number is valid. In this example, it checks whether the phone number consists only of digits and has a length of 10.

**_is_valid_email(self, email_address):**

This private method checks if the provided email address is valid. In this example, it simply checks whether the email address contains the '@' symbol.

**class Person:**

This class represents a person object with attributes for name and contact information.

**contact_info = ContactInformation("contact_info")**

This class attribute defines a descriptor for the `contact_info` attribute of the `Person` class. It creates an instance of the `ContactInformation` class and associates it with the `contact_info` attribute.

**__init__(self, name, contact_info):**

The constructor initializes a `Person` object with the given name and contact information. It takes the person's name and an optional contact information dictionary as arguments.

#  1. Lazily Computed Property

**Purpose of Lazily Computed Property:**
   - Lazily computed properties are used to improve performance by avoiding unnecessary calculations until the property is actually needed.

# 2. Example

The `lazyproperty` decorator allows you to create properties that are computed only once and then cached for future use. Here's a breakdown of the code:


1. **`lazyproperty` Class:**
   - This is a custom class acting as a descriptor.
   - It takes a function (`func`) during initialization, which is assumed to be a method of the class that will be decorated.

2. **`__init__` Method:**
   - The `__init__` method stores the function passed to it (the method to be lazily evaluated).

3. **`__get__` Method:**
   - The `__get__` method is called when the decorated property is accessed.
   - The `if instance is None:` check is necessary because the `__get__` method can also be called when the property is being accessed on the class itself, rather than on an instance of the class. In this case, the instance parameter will be None.
      - If accessed on the class itself (`instance is None`), it returns the `lazyproperty` instance.
      - If accessed on an instance, it computes the value using the stored function and sets the attribute on the instance with the computed value. It then returns the computed value.

4. **Usage in the `Circle` Class:**
   - The `Circle` class has two properties: `area` and `perimeter`, both decorated with `@lazyproperty`.
   - The `area` and `perimeter` properties are computed lazily, meaning that the calculations are performed only when the properties are first accessed, and the results are cached for subsequent accesses.

5. **Example Usage:**
   - Accessing the `area` and `perimeter` properties triggers the lazy computation, and the results are printed.
   - Subsequent accesses to the properties reuse the previously computed values without recalculating.

This approach is a form of memoization, where the results of expensive function calls are cached to avoid redundant computations. It's particularly useful for properties or methods that involve heavy calculations but might not always be needed.

In [7]:
class lazyproperty:
    def __init__(self, func) -> None:
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [8]:
import math

class Circle:
    def __init__(self, radius) -> None:
        self.radius = radius

    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

In [9]:
circle_instance = Circle(radius=5)

In [10]:
circle_instance.area

Computing area


78.53981633974483

In [11]:
# Accessing the area propety again reuses the previously computed value. No computation
circle_instance.area

78.53981633974483

In [12]:
circle_instance.perimeter

Computing perimeter


31.41592653589793

In [13]:
# Accessing the perimeter propety again reuses the previously computed value. No computation
circle_instance.perimeter

31.41592653589793